#### Importação Bibliotecas/funções

In [1]:
# Importação das bibliotecas necessárias para trabalhar com o PySpark
from pyspark.sql import SparkSession  # Importa a classe SparkSession para criar uma sessão Spark
from pyspark.sql.functions import *   # Importa todas as funções disponíveis no módulo pyspark.sql.functions
from pyspark.sql.types import *       # Importa todos os tipos de dados disponíveis no módulo pyspark.sql.types


#### Criar/iniciar Sessão PySpark

In [2]:
# Criação e inicialização da sessão Spark
spark = (
    SparkSession.builder 
    .master('local')                 # Define o modo de execução como "local"
    .appName('PySpark_01')           # Define o nome da aplicação Spark como "PySpark_01"
    .getOrCreate()                   # Cria uma nova sessão Spark ou recupera uma existente se já houver
)

24/05/03 13:54:30 WARN Utils: Your hostname, MacBook-Air-de-Allan.local resolves to a loopback address: 127.0.0.1; using 192.168.99.115 instead (on interface en0)
24/05/03 13:54:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 13:54:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Criar DF / ler arquivo

In [3]:
# Carrega os dados do arquivo 'wc2018-players.csv' para um DataFrame Spark.
# O parâmetro 'header=True' indica que a primeira linha contém o cabeçalho das colunas.
# O parâmetro 'inferSchema=True' permite ao Spark inferir automaticamente o esquema dos dados.


df = spark.read.csv('wc2018-players.csv', header=True, inferSchema=True)



#### Exibir DF

In [4]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



#### Verificar tipos de colunas

In [5]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



#### Verificando dados nulos


In [6]:
df.toPandas().isna().sum()          #Se for um dataframe pequeno

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

In [7]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Team 0
# 0


AnalysisException: Syntax error in attribute name: Pos..

#### Renomeando Colunas

In [8]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Shirt Name', 'Nome Camiseta').withColumnRenamed('Club', 'Time').withColumnRenamed('Height', 'Altura')\
.withColumnRenamed('Weight', 'Peso')

df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [9]:
df.columns

['Selecao',
 'Numero',
 'Posicao',
 'Nome_FIFA',
 'Nascimento',
 'Nome Camiseta',
 'Time',
 'Altura',
 'Peso']

In [10]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


#### Selecionar Colunas

In [11]:
df.select('Selecao', 'Nome_FIFA').show(5)

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [12]:
df.select(col('Selecao'), col('Altura')).show(5)

+---------+------+
|  Selecao|Altura|
+---------+------+
|Argentina|   169|
|Argentina|   169|
|Argentina|   167|
|Argentina|   167|
|Argentina|   170|
+---------+------+
only showing top 5 rows



In [13]:
df.select(df['Selecao']).show(5)

+---------+
|  Selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



#### Selecionar Colunas com *ALIAS*
Um ALIAS é um apelido ou nome alternativo dado a uma coluna em um DataFrame Spark.
É útil quando você deseja renomear temporariamente uma coluna para facilitar a referência
em operações subsequentes, como seleção, agregação ou junção de DataFrames.

**Exemplo de uso de ALIAS:**
Suponha que você tenha um DataFrame df com coluna 'nome' e deseja calcular o comprimento
da coluna 'nome' e renomear a nova coluna como 'comprimento_nome':
**df_comprimento = df.select(length(df['nome']).alias('comprimento_nome'))**

Neste exemplo, 'length(df['nome'])' calcula o comprimento da string na coluna 'nome',
e 'alias('comprimento_nome')' renomeia a coluna resultante para 'comprimento_nome'.
Agora você pode-se referir à nova coluna pelo ALIAS 'comprimento_nome' em operações subsequentes.



In [16]:
df.select(col('Selecao').alias('Time')).show(5)

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



In [17]:
df.select('Selecao Nome_FIFA Altura'.split()).show(5)

+---------+------------------+------+
|  Selecao|         Nome_FIFA|Altura|
+---------+------------------+------+
|Argentina|TAGLIAFICO Nicolas|   169|
|Argentina|    PAVON Cristian|   169|
|Argentina|    LANZINI Manuel|   167|
|Argentina|    SALVIO Eduardo|   167|
|Argentina|      MESSI Lionel|   170|
+---------+------------------+------+
only showing top 5 rows



#### Organizar *Select*